# Library Import

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
import fasttext
from gensim.models.wrappers.fasttext import FastText
import urllib, os
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
print("lib load done")

lib load done


In [2]:
vector_size = 300
encode_length = 6
label_size = 10
vector_model = None

# Parameters
learning_rate = 0.001
training_iters = 100
display_step = 5
batch_size = 5

# Network Parameters
n_input = vector_size
n_hidden = 128 
n_steps = encode_length
n_classes = label_size 
state_parm = None
print("set configuration parms")

set configuration parms


# Download pretrained Facebook fasttext vec for Korean
- https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

In [3]:
def download_fasttext_vec() :
    url = "https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ko.vec"
    file_name = url.split('/')[-1]
    u = urllib.request.urlopen(url)
    if (os.path.exists('./model/' + file_name) == False) : 
        f = open('./model/' + file_name, 'wb')
    else: 
        print("Vector file already exists")
        return False
    meta = u.info()
    file_size = 2000000000
    print ("Downloading: %s Bytes: %s" % (file_name, file_size) )

    file_size_dl = 0
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break

        file_size_dl += len(buffer)
        f.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
        status = status + chr(8)*(len(status)+1)
        if (file_size_dl % 10000000 == 0) :
            print (status)
    f.close()
    print("Download file down")
    return True

# Load FastText Model 

In [4]:
def load_embed_model(vector_model) : 
    if(vector_model == None) : 
        vector_model = FastText.load_word2vec_format('./model/wiki.ko.vec')
    print("Load Vector Model done")
    return vector_model

# Load Train Data

In [5]:
def load_train_csv(model, batch_size):
    df_csv_read = pd.read_csv('./data/train.csv',
                              skipinitialspace=True,
                              engine="python",
                              encoding='utf-8-sig')

    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    
    inputs = []
    labels = []
    
    for encode_raw in df_csv_read['encode'] :   
        encode_raw = mecab.pos(encode_raw)
        encode_raw = list(filter(lambda x :  x[1] in ['MM', 'NNG', 'NNP', 'SF','XSN','NNBC', 'VV', 'NP','VCN+EF'], encode_raw ))
        encode_raw = ''.join(list(map(lambda x : '@' if x[1] in ['SF'] else x[0], encode_raw))).split('@')
        encode_raw = list(map(lambda x : mecab.morphs(x) , encode_raw))
        
        input = np.zeros((1,vector_size),dtype=float)
        for sent in encode_raw : 
            if(len(sent) > 0 ) :
                input = np.array(list(map(lambda x : model[x] if x in model.index2word else np.zeros(vector_size,dtype=float) , sent)))
            if (encode_length - np.shape(input)[0] > 0 ) : 
                input = np.concatenate((input, np.zeros((encode_length - np.shape(input)[0],vector_size),dtype=float)))
            else :
                input = input[0:encode_length]
        inputs.append(input)
        
    for decode_raw in df_csv_read['decode'] : 
        label = np.zeros(label_size, dtype=float)
        np.put(label, decode_raw, 1)
        labels.append(label)

    print("Load data for batch size : {0}".format(batch_size))
    return np.array(inputs), np.array(labels)
    #return np.take(np.array(inputs),np.random.choice(len(inputs),batch_size)) , np.take(np.array(labels),np.random.choice(len(labels),batch_size))

# Build Graph

In [6]:
def build_graph() :
    tf.reset_default_graph()
    # tf Graph input
    x_input = tf.placeholder("float", [None, n_steps, n_input])
    y = tf.placeholder("float", [None, n_classes])

    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x_input,n_steps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    istate = lstm_cell.zero_state(batch_size, tf.float32)
    outputs, states = rnn.static_rnn(lstm_cell, x, initial_state=istate, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    pred = tf.matmul(outputs[-1], weights['out']) + biases['out']

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    pred_result = tf.argmax(pred,1)
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()
    print("Build graph done")
    
    return x_input,y,pred, cost,init, optimizer, accuracy, pred_result, lstm_cell, states, istate

# Run Train

In [7]:
def train():
    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.Saver(tf.all_variables())
        step = 1

        # Keep training until reach max iterations
        while step * batch_size < training_iters:
            batch_x, batch_y = load_train_csv(vector_model, batch_size)
            # Reshape data to get 28 seq of 28 elements
            batch_x = batch_x.reshape((batch_size, n_steps, n_input))
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x_input: batch_x, y: batch_y})

            if step % display_step == 0:
                # Calculate batch accuracy
                acc = sess.run(accuracy, feed_dict={x_input: batch_x, y: batch_y})
                # Calculate batch loss
                loss = sess.run(cost, feed_dict={x_input: batch_x, y: batch_y})
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
            step += 1
        print("Optimization Finished!")

        # Calculate accuracy for 128 mnist test images
        test_len = 128
        test_data = batch_x
        test_label = batch_y
        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={x_input: test_data, y: test_label}))
        saver.save(sess, './model/')

# Train Process

In [8]:
#fast text word embedding 을 위한 pretrained 된 vector 파일을 다운로드 
download_fasttext_vec()
#Vector 모델을 fasttext 라이브러리에 로드
vector_model = load_embed_model(vector_model)
#그래프를 빌드한다. 
batch_size = 5
x_input,y,pred, cost,init, optimizer, accuracy, lstm_cell, pred_result, state, istate = build_graph()
#실제 트레인을 실행한다 
train()

Downloading: wiki.ko.vec Bytes: 2000000000
 640000000  [32.00%]
1280000000  [64.00%]
1920000000  [96.00%]
Download file down
Load Vector Model done
Build graph done
Instructions for updating:
Please use tf.global_variables instead.
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Iter 25, Minibatch Loss= 0.794948, Training Accuracy= 1.00000
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Iter 50, Minibatch Loss= 0.071836, Training Accuracy= 1.00000
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size : 5
Iter 75, Minibatch Loss= 0.002244, Training Accuracy= 1.00000
Load data for batch size : 5
Load data for batch size : 5
Load data for batch size :

# Predict Data Prepare 

In [9]:
def get_vector(query, model) :
    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    inputs = []
    encode_raw = mecab.pos(query)
    encode_raw = list(filter(lambda x :  x[1] in ['MM', 'NNG', 'NNP', 'SF','XSN','NNBC', 'VV', 'NP','VCN+EF'], encode_raw ))
    encode_raw = ''.join(list(map(lambda x : '@' if x[1] in ['SF'] else x[0], encode_raw))).split('@')
    encode_raw = list(map(lambda x : mecab.morphs(x) , encode_raw))
    print("====morph data : {0}".format(encode_raw))
    input = np.zeros((1,vector_size),dtype=float)
    for sent in encode_raw : 
        if(len(sent) > 0 ) :
            input = np.array(list(map(lambda x : model[x] if x in model.index2word else np.zeros(vector_size,dtype=float) , sent)))
        if (encode_length - np.shape(input)[0] > 0 ) : 
            input = np.concatenate((input, np.zeros((encode_length - np.shape(input)[0],vector_size),dtype=float)))
        else :
            input = input[0:encode_length]
    inputs.append(input)
    return np.array(inputs)
print("done prepare get vector function for prediction")

done prepare get vector function for prediction


# Predict Function

In [10]:
def predict(vector, pred_result, state, state_parm, x_input, lstm_cell, batch_size ) : 
    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.Saver(tf.all_variables())
        saver.restore(sess, './model/')
        if(state_parm == None) : 
            print("=======initialze======")
            result,state_parm = sess.run([pred, state], feed_dict={x_input: vector})    
        else :
            print("=======load state======")
            result,state_parm = sess.run([pred, state], feed_dict={x_input: vector, istate:state_parm})    
    return result, state_parm
print("done prepare predict session run function")

done prepare predict session run function


# Service Prepare

In [11]:
#fast text word embedding 을 위한 pretrained 된 vector 파일을 다운로드 
download_fasttext_vec()
#Vector 모델을 fasttext 라이브러리에 로드
vector_model = load_embed_model(vector_model)
#그래프를 빌드한다. 
batch_size = 1
state_parm = None
x_input,y,pred, cost,init, optimizer, accuracy, lstm_cell, pred_result, state, istate   = build_graph()

Vector file already exists
Load Vector Model done
Build graph done


# Service Test
# [시나리오]
- (의도 : 1) 안녕
- (의도 : 2) 넌 뭐할줄 아냐?
- (의도 : 3) 사람을 찾고싶어
- (의도 : 3) 김승우 찾아줘
- (의도 : 4) 문자보내줘

In [12]:
#실제 트레인을 실행한다 
result, state_parm = predict(get_vector("넌뭐할줄아냐?", vector_model), pred_result, state, state_parm, x_input, lstm_cell, batch_size)
print(state_parm[0])
print("=============================================")
print("Intent Result : {0}".format(np.argmax(result)))
print("=============================================")

====morph data : [['뭐', '아냐'], []]
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from ./model/
=======initialze======
[[ 0.95854872  0.00311426  0.47420675  0.81543148 -1.10605061 -0.70528424
  -0.04886642 -0.47452116 -0.55120313 -0.25828254  0.56813639  0.77732325
  -0.47374451  0.36261284 -0.0922368   0.3239626  -0.14634919 -0.48350111
   0.51146436  0.23472466  0.27762547  0.0541451  -0.15378116  0.10287026
  -0.46569183  0.6232999   0.2875618  -0.16551121  0.06420634  0.00370907
  -0.06195594  0.07124117 -0.09534165 -0.05522212  0.3868804   0.05430844
   0.66743159 -0.1548849   0.81005263  0.13730553 -0.25688592  0.1057874
  -0.16586143 -0.15877753 -0.44768041  0.61756134  0.25059211  0.13127139
  -0.64165598  0.13816799 -0.3383362  -0.39154503  0.34258851 -0.87412286
   0.79840589 -0.6162501  -0.92601758  1.41798806 -0.127694    0.55242479
   0.2979663  -0.43040991 -0.12030718  0.27243057 -0.71002322 -0.72835827
   0.291009

# Reset State

In [13]:
state_parm = None